In [27]:
exchange_info = {
    "LN": {
        "start": "09:30",
        "end": "17:30",
        "exchange_rate": 1.2853,
        "region": "WE",
        "name": "London"
    },
    "US": {
        "start": "13:30",
        "end": "21:30",
        "exchange_rate": 1,
        "region": "NA",
        "name": "United States"
    }
}

In [28]:
exchange_info

{'LN': {'start': '09:30',
  'end': '17:30',
  'exchange_rate': 1.2853,
  'region': 'WE',
  'name': 'London'},
 'US': {'start': '13:30',
  'end': '21:30',
  'exchange_rate': 1,
  'region': 'NA',
  'name': 'United States'}}

In [29]:
exchange_map = {}
for key, value in exchange_info.items():
    exchange_map[key] = value["region"]

In [30]:
exchange_map

{'LN': 'WE', 'US': 'NA'}